In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product

from backtesterClass.orderClass import orders
from backtesterClass.orderBookClass import OBData
from backtesterClass.analysisClass import analysisClass

from strats.basicStrat import basicStrat
from strats.movingAverageStrat import movingAverageStrat
from strats.rsiStrat import rsiStrat
from strats.momentumStrat import momentumStrat
from strats.momentumOnlineLearn import momentumOnlineLearnStrat
from strats.LTSMOnlineLearn import LTSMOnlineLearnStrat

from scipy.stats import spearmanr, pearsonr

from cProfile import Profile
from pstats import SortKey, Stats
import gc

In [2]:
# Improve computationnal performance of the backtester - increases garbage collector threshold

allocs, gen1, gen2 = gc.get_threshold()
allocs = 5000
gen1 = gen1*2
gen2=gen2*2
gc.set_threshold(allocs, gen1,gen2)

In [3]:
db_path = r"backtesterClass/stockDB.db"

dataClass = OBData(db_path)

In [4]:
autoTrader = basicStrat("autoTrader")
movingAverageTrader = movingAverageStrat("movingAverageTrader", short_window = 20, long_window=200)
rsiTrader = rsiStrat("rsiTrader", window=50, buyThreshold=30, sellThreshold=70, alpha=0.05)
momentumTrader = momentumStrat("momentumTrader", short_window = 20, long_window=200, RSI_window=50, sellThreshold=70,buyThreshold=30, alpha=0.05)
momentumOnlineTrader = momentumOnlineLearnStrat("momentumOnlineTrader", short_window = 20, long_window=200, RSI_window=50, sellThreshold=70,buyThreshold=30, alpha=0.05 ,trainLen = 200, forecast=1)
LTMSOnlineTrader = LTSMOnlineLearnStrat("LTSMOnlineTrader", short_window = 20, long_window=200, RSI_window=50, sellThreshold=70,buyThreshold=30, alpha=0.05 ,trainLen = 200, forecast=1)

['cumulative returns', 'lag_ret_1', 'lag_ret_10', 'lag_ret_2', 'lag_ret_5', 'long_ma', 'rsi', 'short_ma'] | StandardScaler | RollingRegressor


In [7]:
for _ in tqdm(range(len(dataClass.OBData_))):
    orderClass = orders()
    # autoTrader.strategy(orderClass)
    movingAverageTrader.strategy(orderClass)
    rsiTrader.strategy(orderClass)
    # momentumTrader.strategy(orderClass)
    # momentumOnlineTrader.strategy(orderClass)
    # LTMSOnlineTrader.strategy(orderClass)
    OBData.step +=1 

100%|██████████| 4243/4243 [00:24<00:00, 171.29it/s]


In [8]:
# analysisBasic = analysisClass(autoTrader ,path=r"/Users/leo/Desktop/EDHEC MsC FENg/Master Project/Project/Results", dashboardName="executionBasicStrat", dbName="dbBasicStrat")
# analysisBasic.create_dashboard(asset="AAPL")
# analysisBasic.dataBase()

In [9]:
analysisMA = analysisClass(movingAverageTrader, path=r"Results", dashboardName="executionMovingAverageStrat", dbName="dbMovingAverageStrat")
analysisMA.create_dashboard(asset="AAPL")
analysisMA.dataBase()

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBA

In [10]:
analysisRSI = analysisClass(rsiTrader, path=r"Results", dashboardName="executionRSIStrat", dbName="dbRSIStrat")
analysisRSI.create_dashboard("AAPL")
analysisRSI.dataBase()

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBA

In [11]:
# analysisMomentum = analysisClass(momentumTrader, path=r"Results", dashboardName="executionMomentumStrat", dbName="dbMomentumStrat")
# analysisMomentum.create_dashboard("AAPL")
# analysisMomentum.dataBase()

In [12]:
# analysisOLMomentum = analysisClass(momentumOnlineTrader, path=r"Results", dashboardName="executionOLMomentumStrat", dbName="dbOLMomentumStrat")
# analysisOLMomentum.create_dashboard("AAPL")
# analysisOLMomentum.dataBase()

In [15]:
analysisLTSM = analysisClass(LTMSOnlineTrader, path=r"Results", dashboardName="executionLTSMStrat", dbName="dbLTSMStrat")
analysisLTSM.create_dashboard("AAPL")
analysisLTSM.dataBase()

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBA